One-off code to turn the CORE sample (a bunch of nested folders) into a JSON 

In [1]:
#https://core.ac.uk/exports/resync_datadump_sample220218.zip
#!unzip 'resync_datadump_sample220218.zip' -d "datadump/"
#!mkdir jsons

In [11]:
tar -x"downloads/140.tar.xz"

SyntaxError: invalid syntax (<ipython-input-11-6fcda8dd4500>, line 1)

In [22]:
for i in `ls {100..300}.tar.xz`
do
	i
done
    
for VARIABLE in 1 2 3 4 5 .. N
do
	command2
	commandN
done

/bin/sh: -c: line 0: syntax error near unexpected token `('
/bin/sh: -c: line 0: `for i in `ls downloads/{100..300}.tar.xz`: print(i)'


In [20]:
!for VARIABLE in 
!do
!	ls VARIABLE
!done

/bin/sh: -c: line 1: syntax error: unexpected end of file
/bin/sh: -c: line 0: syntax error near unexpected token `do'
/bin/sh: -c: line 0: `do'
ls: cannot access VARIABLE: No such file or directory
/bin/sh: -c: line 0: syntax error near unexpected token `done'
/bin/sh: -c: line 0: `done'


In [1]:
from tqdm import tqdm


In [28]:
import os
import json

locs = []
#150 is pubmed apparently


k =0
for root, dirs, files in tqdm(os.walk("downloads/pubtest", topdown=False)):
#     k+= 1
#     if k > 500000:
#         break
    for name in files:    
        fname = os.path.join(root, name)
        if (fname.endswith(".json")):
            locs.append(fname)


#locs[0]


935944it [00:13, 68321.36it/s]


In [29]:
len(locs)

2305355

In [14]:
#ls downloads/140

00/  13/  22/  34/  45/  56/  67/  76/  86/  98/  a9/  b9/  c9/  d9/  e9/  fc/
01/  14/  23/  35/  46/  57/  68/  77/  87/  99/  aa/  bb/  ca/  da/  ea/  fd/
02/  15/  24/  36/  47/  59/  69/  78/  88/  9a/  ab/  bc/  cb/  db/  eb/  fe/
04/  16/  25/  37/  49/  5a/  6a/  79/  8b/  9b/  ac/  bd/  cc/  dc/  ec/  ff/
05/  17/  26/  38/  4a/  5b/  6b/  7a/  8c/  9c/  ad/  be/  cd/  dd/  ed/
06/  18/  28/  3a/  4b/  5c/  6c/  7b/  8d/  9e/  ae/  bf/  ce/  de/  ee/
08/  19/  29/  3b/  4c/  5d/  6d/  7c/  8e/  9f/  af/  c0/  cf/  df/  ef/
09/  1a/  2a/  3c/  4e/  5e/  6e/  7d/  90/  a1/  b0/  c1/  d0/  e0/  f0/
0a/  1b/  2b/  3d/  4f/  5f/  6f/  7e/  91/  a2/  b2/  c2/  d1/  e1/  f2/
0b/  1c/  2c/  3e/  50/  60/  70/  7f/  92/  a3/  b3/  c3/  d2/  e2/  f3/
0c/  1d/  2f/  3f/  51/  62/  71/  80/  93/  a4/  b4/  c4/  d3/  e3/  f4/
0e/  1e/  30/  40/  52/  63/  72/  82/  94/  a5/  b5/  c5/  d4/  e4/  f5/
0f/  1f/  31/  41/  53/  64/  73/  83/  95/  a6/  b6/  c6/  d5/  e5/  f7/
10/  20/  32/  43/

In [30]:
#d
jsons = []

for loc in tqdm(locs):
    f = open(loc, "r")
    data = json.load(f)
    jsons.append(data)
d = {"data": jsons}

100%|██████████| 2305355/2305355 [06:14<00:00, 6155.30it/s] 


In [31]:
pubmed = [j for j in d["data"] if "pubmed" in j["oai"] and j["abstract"]]
    

In [32]:
df = pd.json_normalize(pubmed)

In [34]:
df2 = df
df2 = df2.drop(columns=["abstract"])

In [36]:
#export

import json
import re


alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [37]:
!pip install pydash



     |████████████████████████████████| 84 kB 507 kB/s             


In [38]:
from pydash import get

In [40]:
df2["first_sent"] = df["abstract"].apply(lambda a: get(split_into_sentences(a), 0))

In [41]:
df2.to_csv("downloads/core_first_sent.csv", index = False, header=True)

In [42]:
df2

,doi,coreId,oai,identifiers,title,authors,contributors,datePublished,downloadUrl,fullTextIdentifier,...,urls,issn,enrichments.references,enrichments.citationCount,enrichments.documentType.type,enrichments.documentType.confidence,language.code,language.name,language.id,first_sent
0,None,8361774,oai:pubmedcentral.nih.gov:2797167,[],αvβ3 Integrin Limits the Contribution of Neuro...,"[Robinson, Stephen D., Reynolds, Louise E., Ko...",[],None,None,http://www.pubmedcentral.nih.gov/articlerender...,...,[http://www.pubmedcentral.nih.gov/articlerende...,None,[],NaN,None,NaN,NaN,NaN,NaN,Both vascular endothelial growth factor recept...
1,10.1136/oem.20.4.277,3602553,oai:pubmedcentral.nih.gov:1038379,[],Energy Expenditure Ranges and Muscular Work Gr...,"[Brown, J. R., Crowden, G. P.]",[],1963-10,None,http://www.pubmedcentral.nih.gov/articlerender...,...,None,1351-0711,[],22.0,None,NaN,NaN,NaN,NaN,This paper is based on the findings of a field...
2,10.1128/mcb.11.4.2282,3428118,oai:pubmedcentral.nih.gov:359930,[],Characterization of a telomere-binding protein...,"[Coren, J S, Epstein, E M, Vogt, V M]",[],1991-04,None,http://www.pubmedcentral.nih.gov/articlerender...,...,None,0270-7306,[],NaN,None,NaN,NaN,NaN,NaN,We have partially purified a nuclear protein (...
3,None,8208292,oai:pubmedcentral.nih.gov:2572022,[],A Fortuitous Syncope. The pitfalls of Integrat...,"[Salukhe, Tushar V, Wright, Ian, Wright, Matth...",[],None,https://core.ac.uk/download/pdf/8208292.pdf,http://www.pubmedcentral.nih.gov/articlerender...,...,[http://www.pubmedcentral.nih.gov/articlerende...,None,"[{'date': '1999', 'raw': 'Babuty D, Fauchier L...",NaN,None,NaN,en,English,9.0,Myopotential oversensing in implantable defibr...
4,None,8364895,oai:pubmedcentral.nih.gov:2801609,[],Control of Alzheimer's Amyloid Beta Toxicity b...,"[Sanokawa-Akakura, Reiko, Cao, Weihuan, Allan,...",[],None,https://core.ac.uk/download/pdf/8364895.pdf,http://www.pubmedcentral.nih.gov/articlerender...,...,[http://www.pubmedcentral.nih.gov/articlerende...,None,"[{'date': '1986', 'raw': 'Tai PK, Maeda Y, Nak...",NaN,None,NaN,en,English,9.0,"FK506 binding proteins (FKBPs), also called im..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1144178,10.1128/jb.170.3.1403-1404.1988,3295288,oai:pubmedcentral.nih.gov:210926,[],Properties of spores of Bacillus subtilis stra...,"[Hackett, R H, Setlow, P]",[],1988-03,None,http://www.pubmedcentral.nih.gov/articlerender...,...,None,0021-9193,[],31.0,None,NaN,NaN,NaN,NaN,Bacillus subtilis strains containing a deletio...
1144179,None,8061207,oai:pubmedcentral.nih.gov:2393453,[],"Health, safe water and sanitation: a cross-sec...","[Wibowo, D., Tisdell, C.]",[],None,None,http://www.pubmedcentral.nih.gov/articlerender...,...,[http://www.pubmedcentral.nih.gov/articlerende...,None,[],NaN,None,NaN,NaN,NaN,NaN,The study describes the development of health ...
1144180,None,3892463,oai:pubmedcentral.nih.gov:1388572,[],Reduction of Nitrated Diphenylamine Derivative...,"[Drzyzga, O., Schmidt, A., Blotevogel, K.]",[],1995-09,None,http://www.pubmedcentral.nih.gov/articlerender...,...,None,None,[],NaN,None,NaN,NaN,NaN,NaN,"2-Nitrodiphenylamine, 4-nitrodiphenylamine, an..."
1144181,None,8585567,oai:pubmedcentral.nih.gov:3123099,[],Active Site-labeled Prothrombin Inhibits Proth...,"[Kroh, Heather K., Panizzi, Peter, Tchaikovski...",[],None,None,http://www.pubmedcentral.nih.gov/articlerender...,...,[http://www.pubmedcentral.nih.gov/articlerende...,None,[],NaN,None,NaN,NaN,NaN,NaN,Mouse and human prothrombin (ProT) active site...


In [33]:
df.to_csv("downloads/core3.csv", index = False, header=True)

## Archived Code Snippets 

In [ ]:
with open('downloads/CORE_out.json') as f:
    g = f.read()
    data = json.loads(g)

In [ ]:
data =d
hits = [paper for paper in data['data'] if paper["abstract"] and paper["fullText"] and len(paper["abstract"])> 15]
len(data["data"]), len(hits)

(21841, 7595)

In [ ]:

#from utils
coll = [{"abstract": e["abstract"], "fullText": e["fullText"], "doi": e["doi"]} for e in hits]
coll[0]
[c.update(
    {"keysentence_abstract": find_sentence_in_abstract(c["abstract"]), 
    "sent_before_ABST":find_sentence_in_abstract(c["abstract"], -1),
    "sent_after_ABST":find_sentence_in_abstract(c["abstract"], -1),

     "keysentence_fulltext": find_sentence_in_abstract(c["fullText"]),
     "sent_before_FT":find_sentence_in_abstract(c["fullText"], -1),
     "sent_after_FT": find_sentence_in_abstract(c["fullText"], 1),
     }) for c in coll]

4

[c.update({"sentiment": sentiment(c["phrase"])}) for c in phrase_hits]
[c.update({"modality": modality(c["phrase"]) }) for c in phrase_hits]

4
only_both = [c for c in coll if c["keysentence_abstract"] and c["keysentence_fulltext"]]
len(only_both)